In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data

from matrix_factorization import NCF, NCF_CVIB, NCF_IPS, NCF_SNIPS, NCF_DR

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"
# dataset_name = "yahoo"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300


In [2]:
"NCF CVIB"
ncf_cvib = NCF_CVIB(num_user, num_item)
ncf_cvib.fit(x_train, y_train, lr=0.05, 
    alpha=1.0, gamma=1e-5, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)

test_pred = ncf_cvib.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(ncf_cvib, x_test, y_test)

print("***"*5 + "[NCF-CVIB]" + "***"*5)
print("[NCF-CVIB] test mse:",mse_ncf)
print("[NCF-CVIB] test auc:", auc_ncf)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-CVIB]" + "***"*5)

[NCF-CVIB] epoch:0, xent:10.383777022361755
[NCF-CVIB] epoch:10, xent:9.018936455249786
[NCF-CVIB] epoch:18, xent:8.790817499160767
***************[NCF-CVIB]***************
[NCF-CVIB] test mse: 0.3045055561048399
[NCF-CVIB] test auc: 0.770635320173312
ndcg@5:0.642075, ndcg@10:0.717487
Num User: 290
Gini index: 0.32342105263157994
Global utility: 0.5241379310344828
***************[NCF-CVIB]***************


In [3]:
"NCF naive"
ncf = NCF(num_user, num_item)
ncf.fit(x_train, y_train, lr=0.05, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)
test_pred = ncf.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(ncf, x_test, y_test)

print("***"*5 + "[NCF]" + "***"*5)
print("[NCF] test mse:", mse_func(y_test, test_pred))
print("[NCF] test auc:", auc_ncf)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF]" + "***"*5)

[NCF] epoch:0, xent:9.007273197174072
[NCF] epoch:10, xent:6.419635742902756
[NCF] epoch:20, xent:6.214524537324905
[NCF] epoch:27, xent:6.196675032377243
***************[NCF]***************
[NCF] test mse: 0.2044858737260562
[NCF] test auc: 0.7624072136527681
ndcg@5:0.614147, ndcg@10:0.692171
Num User: 290
Gini index: 0.3402174923320018
Global utility: 0.5117241379310344
***************[NCF]***************


In [4]:
"NCF IPS"
ncf_ips = NCF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_ips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)

ndcg_res = ndcg_func(ncf_ips, x_test, y_test)
test_pred = ncf_ips.predict(x_test)
mse_ncfips = mse_func(y_test, test_pred)
auc_ncfips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-IPS]" + "***"*5)
print("[NCF-IPS] test mse:", mse_ncfips)
print("[NCF-IPS] test auc:", auc_ncfips)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-IPS]" + "***"*5)

[NCF-IPS] epoch:0, xent:115.08175849914551
[NCF-IPS] epoch:10, xent:80.50548887252808
[NCF-IPS] epoch:20, xent:73.78963422775269
[NCF-IPS] epoch:30, xent:70.84964084625244
[NCF-IPS] epoch:40, xent:66.75966358184814
[NCF-IPS] epoch:50, xent:63.60177230834961
[NCF-IPS] epoch:60, xent:61.39280033111572
[NCF-IPS] epoch:61, xent:61.41753673553467
***************[NCF-IPS]***************
[NCF-IPS] test mse: 0.21151629583206863
[NCF-IPS] test auc: 0.7600945630042399
ndcg@5:0.616355, ndcg@10:0.696604
Num User: 290
Gini index: 0.3409207879099808
Global utility: 0.5082758620689656
***************[NCF-IPS]***************


In [5]:
"NCF SNIPS"
ncf_snips = NCF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_snips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)
ndcg_res = ndcg_func(ncf_snips, x_test, y_test)
test_pred = ncf_snips.predict(x_test)
mse_ncfsnips = mse_func(y_test, test_pred)
auc_ncfsnips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)
print("[NCF-SNIPS] test mse:", mse_ncfsnips)
print("[NCF-SNIPS] test auc:", auc_ncfsnips)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)


[NCF-SNIPS] epoch:0, xent:9.005004107952118
[NCF-SNIPS] epoch:10, xent:7.94594407081604
[NCF-SNIPS] epoch:20, xent:6.2902533411979675
[NCF-SNIPS] epoch:30, xent:6.2653875052928925
[NCF-SNIPS] epoch:35, xent:6.294830828905106
***************[NCF-SNIPS]***************
[NCF-SNIPS] test mse: 0.1935660983974304
[NCF-SNIPS] test auc: 0.7703781012234381
ndcg@5:0.632121, ndcg@10:0.708180
Num User: 290
Gini index: 0.3407732219110161
Global utility: 0.5179310344827587
***************[NCF-SNIPS]***************


In [6]:
"NCF DR"
ncf_dr = NCF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_dr.fit(x_train, y_train, y_ips=y_ips, batch_size=512, lr=0.01, lamb=1e-3,verbose=0)
ndcg_res = ndcg_func(ncf_dr, x_test, y_test)

test_pred = ncf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-DR]" + "***"*5)
print("[NCF-DR] test mse:", mse_mfdr)
print("[NCF-DR] test auc:", auc_mfdr)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-DR]" + "***"*5)

D:\Counterfactual\CVIB-Rec\matrix_factorization.py:1125: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  xent_loss = F.binary_cross_entropy(pred, sub_y, weight=inv_prop, reduction="sum")


[NCF-DR] epoch:61, xent:30060.72021484375
***************[NCF-DR]***************
[NCF-DR] test mse: 0.22631632916739772
[NCF-DR] test auc: 0.746230935252355
ndcg@5:0.611002, ndcg@10:0.683358
Num User: 290
Gini index: 0.34770824464311145
Global utility: 0.5027586206896553
***************[NCF-DR]***************
